<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#en-X86" data-toc-modified-id="en-X86-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>en X86</a></span></li><li><span><a href="#Trouver-la-faille-?" data-toc-modified-id="Trouver-la-faille-?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Trouver la faille ?</a></span></li><li><span><a href="#Solution" data-toc-modified-id="Solution-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Solution</a></span></li><li><span><a href="#Comment-l'éviter-?" data-toc-modified-id="Comment-l'éviter-?-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Comment l'éviter ?</a></span></li></ul></div>

# BufferOverflow

Le problème du dépassement de tampon (ou Buffer Overflow, ou BOF) est une faille de sécurité intervenant quand une saisie dépasse la taille prévue et déborde sur le reste de la mémoire. En général, cela fait "juste" planter le programme, mais si la faille est mal placée et qu'un attaquant cherche à l'exploiter, cela peut devenir ennuyeux.
Pour en savoir plus : 

En général, la donnée saisie est un chaîne de caractères prévue pour une zone mémoire de taille fixe.

Pour donner un exemple, prenons un cas de programme C qui attend une chaine de caractères et vérifie que cette chaine correspond bien à un nom connu (ici "John"). Ce programme, simple, doit écrire "Bonjour" si le nom est bon (et rien ou "Pardon ?" sinon). 

In [1]:
%%writefile bonjourOverflow.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

static char EntreeLue[10];
static char ReponseAttendu[10];

int main (int argc, char * argv[]) {
    strcpy(ReponseAttendu, "John");
    if (argc==1) {
        printf("Il faut un argument\n");}
    else {
        strcpy(EntreeLue, argv[1]);        
        if (strncmp(ReponseAttendu,EntreeLue,10)==0) {printf("Bonjour\n");}
        else {printf("Pardon ?\n");}}
    return 0;}

Overwriting bonjourOverflow.c


In [2]:
%%sh
arm-linux-gnueabi-gcc -S -static bonjourOverflow.c
cat bonjourOverflow.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"bonjourOverflow.c"
	.local	EntreeLue
	.comm	EntreeLue,10,4
	.local	ReponseAttendu
	.comm	ReponseAttendu,10,4
	.section	.rodata
	.align	2
.LC0:
	.ascii	"John\000"
	.align	2
.LC1:
	.ascii	"Il faut un argument\000"
	.align	2
.LC2:
	.ascii	"Bonjour\000"
	.align	2
.LC3:
	.ascii	"Pardon ?\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	str	r0, [fp, #-8]
	str	r1, [fp, #-12]
	ldr	r3, .L6
	ldr	r2, .L6+4
	ldmia	r2, {r0, r1}
	str	r0, [r3, #0]
	add	r3, r3, #4
	strb	r1, [r3, #0]
	ldr	r3, [fp, #-8]
	cmp	r3, #1
	bne	.L2
	ldr	r0, .L6+8
	bl	puts
	b	.L3
.L2:
	ldr	r3, [fp, #-12]
	add	r3, r3, #4
	ldr	r3, [r3, #0]
	ldr

In [6]:
%%sh
arm-linux-gnueabi-gcc -static bonjourOverflow.c
qemu-arm a.out JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ

Bonjour


In [39]:
%%sh
arm-linux-gnueabi-gcc -static bonjourOverflow.c
qemu-arm a.out Tom

Pardon ?


Le fonctionnement prévu est le suivant :
* pour John, il dit "Bonjour"
* pour Joe (ou tout le reste), il dit "Pardon ?"


Vous voulez essayer ? (le jeu, maintenant, c'est de trouver comment lui faire dire bonjour, sans dire John.)

In [43]:
%%sh
arm-linux-gnueabi-gcc -static bonjourOverflow.c
qemu-arm a.out Joe

Pardon ?


## en X86

In [47]:
%%sh
gcc bonjourOverflow.c
./a.out Tom

Pardon ?


In [48]:
%%sh
gcc bonjourOverflow.c
./a.out John

Bonjour


Le fonctionnement prévu est le suivant :
* pour John, il dit "Bonjour"
* pour Joe (ou tout le reste), il dit "Pardon ?"

## Trouver la faille ?

Le jeu, maintenant, c'est de trouver comment lui faire dire bonjour, sans dire John.

In [37]:
%%sh
gcc bonjourOverflow.c
./a.out abcdefghijabcdefghij

Bonjour


## Solution

La Solution ?  

Une solution ?

Vous voulez la solution ?

Vous voulez une solution ?

Toute la solution ?

Juste un morceau ?

Il s'agit juste de déborder de EntreeLue pour écrire sur ReponseAttendu pour que les 2 chaînes se ressemblent (la comparaison ne porte que sur 10 caractères.)

On peut essayer de mettre une chaîne de 10 caractères qui se répète (mais il faut bien évaluer l'espace entre la chaîne initiale et la répétition (mais on y arrive !)

Mais il y a plus simple.

Le plus simple est de mettre une chaîne avec un seul caractère et de le répéter au moins 20 fois, mais probablement un peu plus que 20 fois, par exemple, une cinquantaine de fois (on doit pouvoir faire moins, il faudra peut-être faire plus parfois)

Et cela marche.

Pour une chaine intelligente (sur la machine X86 de l'essai : abcdefghijxxxxxxabcdefghij, sur la machine arm émulée abcdefghijxxabcdefghij, ailleurs encore abcdefghijabcdefghij)

Pour une chaîne stupide (partout ça marche avec : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx)

## Comment l'éviter ?

Ne pas utiliser scanf, strcmp, strcpy, etc. sans limiter la taille de la saisie ou du test ou de la recopie ou etc. Par exemple, ici utiliser strncpy.

In [7]:
%%writefile bonjourOverflowCorrigee.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

static char EntreeLue[10];
static char ReponseAttendu[10];

int main (int argc, char * argv[]) {
    strncpy(ReponseAttendu, "John",10);
    if (argc==1) {
        printf("Il faut un argument\n");}
    else {
        strncpy(EntreeLue, argv[1],10);        
        if (strncmp(ReponseAttendu,EntreeLue,10)==0) {printf("Bonjour\n");}
        else {printf("Pardon ?\n");}}
    return 0;}

Writing bonjourOverflowCorrigee.c


In [8]:
%%sh
gcc bonjourOverflowCorrigee.c
./a.out xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Pardon ?
